In [ ]:
import cv2
import timeit
import tensorflow as tf
from ultralytics import YOLO
import numpy as np

# TFLite 모델 파일 경로
model_path = 'C:/Users/USER/Documents/vision-AI/project/7899/models/saved_model/best_float32.tflite'

# TFLite 모델을 로드합니다.
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# 입력 텐서 정보 가져오기
input_details = interpreter.get_input_details()

# 출력 텐서 정보 가져오기
output_details = interpreter.get_output_details()

# 입력 텐서의 모양 및 데이터 유형
input_shape = input_details[0]['shape']
input_dtype = input_details[0]['dtype']

# 출력 텐서의 모양 및 데이터 유형
output_shape = output_details[0]['shape']
output_dtype = output_details[0]['dtype']


#웹캠 초기화
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 알고리즘 시작 시점
    start_t = timeit.default_timer()

    annotated_frame = frame  # 기본적으로 원본 프레임으로 초기화

    # 모델을 사용하여 프레임에서 객체 감지
    # 입력 데이터 크기를 [1, 3, 640, 640]로 변경
    input_data = np.zeros((1,640, 640, 3), dtype=np.uint8)
    for i in range(480):
        for j in range(640):
            for rgb in range(3):
                input_data[0][i][j][rgb] = frame[i][j][rgb]

    input_data = input_data.astype(np.float32)  # FLOAT32로 변환

    # 입력 텐서에 데이터를 설정합니다.
    interpreter.set_tensor(input_details[0]['index'], input_data)
    
    # 추론을 실행합니다.
    interpreter.invoke()

    # 출력 텐서에서 결과 가져오기
    output_data = interpreter.get_tensor(output_details[0]['index'])


    # 출력을 해석합니다.
    # (1, 6, 8400) -> (batch_size, attributes, num_detections)
    detections = output_data[0]  # (6, 8400)

    print(detections[0])

    for detection in detections.T:  # Transpose to iterate over detections
        score = detection[4]  # 예시로 5번째 요소가 신뢰도(score)라고 가정
        if score > 0.8:  # 임계값(Threshold) 설정
            x, y, w, h = detection[:4]
            class_id = detection[5]
            # 바운딩 박스 그리기
            left = int((x - w / 2) * annotated_frame.shape[1])
            top = int((y - h / 2) * annotated_frame.shape[0])
            right = int((x + w / 2) * annotated_frame.shape[1])
            bottom = int((y + h / 2) * annotated_frame.shape[0])
            cv2.rectangle(annotated_frame, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(annotated_frame, f'Class: {class_id}, Score: {score:.2f}', 
                        (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, 
                        (0, 255, 0), 2)
    print(detections.T[0])

    # 알고리즘 종료 시점
    terminate_t = timeit.default_timer()
    
    FPS = int(1./(terminate_t - start_t ))
    cv2.imshow('video',frame)
    print(FPS)

    # 화면에 표시
    cv2.imshow("YOLOv8 Real-Time Detection", annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()